# Tutorial: Controlling Agent Teams with Termination Conditions

Termination conditions are rules that tell an agent team when to stop working. They're essential for creating controlled, interactive workflows instead of letting a conversation run indefinitely.

This tutorial will cover two primary types of termination:
1.  **`TextMentionTermination`**: Stopping when a specific keyword is mentioned.
2.  **`HandoffTermination`**: Stopping when one agent needs to "hand off" control to another.

## The Scenario: A Collaborative Writing Team ✍️

We'll create a team with two AI agents:
1.  **`storyteller_agent`**: Writes a short story.
2.  **`editor_agent`**: Reviews the story, provides feedback, and sends it back for revisions.

This process will loop until the `editor_agent` is satisfied and formally approves the story. We'll use termination conditions to manage the back-and-forth collaboration between them.

### Workflow Visualization

The following flowchart shows the iterative loop between the storyteller and the editor, with a final exit condition.

```mermaid
%% This defines the flowchart direction from Top to Down.
graph TD

    %% Define the nodes (shapes) of the flowchart.
    A([Start: User provides a story prompt])
    B[storyteller_agent: Writes Draft]
    C[editor_agent: Reviews Draft]
    D{Is the story approved?}
    E([End: Collaboration Complete])

    %% Define the main workflow path.
    A --> B
    B -- "Handoff to Editor" --> C
    C --> D

    %% Define the decision paths, including the feedback loop.
    D -- "No, needs revision.<br/>(Handoff to Storyteller)" --> B
    D -- "Yes<br/>(Responds 'STORY APPROVED')" --> E
```

### 1. Setting Up the Agents with Handoffs

First, we'll import the necessary libraries and set up our agents. The key is the **`handoffs`** parameter, which creates a special tool an agent can use to pass control to another agent. We'll also give each agent a system message that guides its behavior.

In [1]:
!pip install --quiet -U "autogen-agentchat>=0.7" "autogen-ext[openai,mcp]>=0.7" "mcp>=1.0.0"

# IMPORTANT: See: https://github.com/microsoft/autogen/issues/6906
!pip install --quiet --force-reinstall "openai==1.80"

> **⚠️ IMPORTANT:**  
> If you just ran the `!pip install --quiet --force-reinstall "openai==1.80"` command,  
> you **must** restart the Jupyter kernel before continuing.  
> This ensures the newly installed `openai` package is loaded into memory  
> and avoids mixed-version issues that can cause runtime errors.  
>  
> **In Jupyter:** go to **Kernel → Restart & Clear Output**, then rerun the notebook from the top.

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import Handoff
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create a model client. Ensure you have your OPENAI_API_KEY environment variable set.
model_client = OpenAIChatCompletionClient(model="gpt-4o")

# Create the Storyteller agent.
storyteller_agent = AssistantAgent(
    "storyteller",
    model_client=model_client,
    system_message="You are a creative storyteller. Write a short, three-sentence story based on the user's prompt. After writing, pass it to the 'editor' for review.",
    # This creates a 'transfer_to_editor' tool.
    handoffs=[Handoff(target="editor", message="Draft complete. Passing to editor for review.")],
)

# Create the Editor agent.
editor_agent = AssistantAgent(
    "editor",
    model_client=model_client,
    system_message="You are a sharp-eyed editor. Review the story from the 'storyteller'. If it needs changes, provide clear feedback and hand it back. If it's perfect, respond with only the words 'STORY APPROVED'.",
    # This creates a 'transfer_to_storyteller' tool.
    handoffs=[Handoff(target="storyteller", message="Edits attached. Returning to storyteller.")],
)

### 2. Defining the Termination Logic

Next, we define the rules for when the team should pause. We need three conditions which we will combine into a single condition using the `|` (OR) operator:
1.  Pause when the storyteller hands off to the editor.
2.  Pause when the editor hands off to the storyteller.
3.  Stop completely when the editor says "STORY APPROVED".

In [3]:
# The team will stop if a handoff to the editor is detected.
handoff_to_editor = HandoffTermination(target="editor")

# The team will stop if a handoff to the storyteller is detected.
handoff_to_storyteller = HandoffTermination(target="storyteller")

# The team will stop if the text "STORY APPROVED" is mentioned.
approval = TextMentionTermination("STORY APPROVED")

# Combine all conditions. The team will stop if ANY of these are met.
termination_logic = handoff_to_editor | handoff_to_storyteller | approval

### 3. Creating and Running the Team

Now we can assemble our team using a `RoundRobinGroupChat`, which cycles through the agents. We'll run the team in a loop, allowing the conversation to continue until the final "STORY APPROVED" condition is met.

In [4]:
# Create the team with our agents and combined termination logic.
writing_team = RoundRobinGroupChat(
    [storyteller_agent, editor_agent],
    termination_condition=termination_logic,
)

# Start the conversation with an initial task.
task = "Write a story about a lost robot in a forest."
result = None

print(f"Starting collaboration for task: '{task}'\n")

# Loop until the final approval is given.
while True:
    # Since we are in a Jupyter Notebook, we can use top-level await.
    result = await writing_team.run(task=task)
    
    # Print the last message for context.
    last_message = result.messages[-1]
    print(f"\n---------- Turn Ended: {last_message.source} ----------")
    print(f"\n{last_message.content}\n----------------------------------------\n")

    # Check if the reason for stopping was the final approval.
    if "STORY APPROVED" in result.stop_reason:
        print("\n✅ Story approved! The collaboration is complete.")
        break
    else:
        print(f" Handoff Detected: {result.stop_reason}. Resuming collaboration...")
        # The next task is empty, allowing the next agent to simply respond to the history.
        task = ""

Starting collaboration for task: 'Write a story about a lost robot in a forest.'


---------- Turn Ended: storyteller ----------

Draft complete. Passing to editor for review.
----------------------------------------

 Handoff Detected: Handoff to editor from storyteller detected.. Resuming collaboration...

---------- Turn Ended: editor ----------

STORY APPROVED
----------------------------------------


✅ Story approved! The collaboration is complete.


### 4. Analyzing the Execution Flow

When you run the code above, you'll see a conversation unfold in turns:

**Round 1: Storyteller to Editor**
The `storyteller_agent` writes a draft and then calls its `transfer_to_editor` tool. The `handoff_to_editor` condition catches this, and the team pauses. The loop prints the `Handoff Detected` message and continues.

**Round 2: Editor to Storyteller**
The `editor_agent` reviews the draft, provides feedback (e.g., "Good start, but make the robot's feelings more apparent."), and calls its `transfer_to_storyteller` tool. The `handoff_to_storyteller` condition catches this, and the team pauses again.

**Further Rounds...**
This back-and-forth continues as the storyteller revises the story based on feedback.

**Final Round: Approval**
Eventually, the `storyteller_agent` incorporates the feedback to the editor's satisfaction. The `editor_agent` then responds with "STORY APPROVED". The `TextMentionTermination` condition catches this, and the `break` statement in our loop stops the collaboration permanently.

This example shows how you can use termination conditions to orchestrate complex, multi-turn workflows between multiple agents, pausing and resuming until a final goal is achieved.